## CUDA Basic Programming

##GPU Learn the basic steps of coding

- CPU Memory Settings
- CPU Memory data settings
- GPU Memory Settings : cudaMalloc(...)
- CPU --> GPU Data Transfer: cudaMemcpy(to, from, sizeofdata, cudaMemcpyHostToDevice)
- GPU Functions (Kernel) Perform
- GPU --> CPU Transfer operation result data: : cudaMemcpy(to, from, sizeofdata, cudaMemcpyDeviceToHost);
- Use the result of the operation on the CPU

*  *  *

# Change the n value and look at the execution time and data delivery time through nvprof.

In [ ]:
%%writefile cudabasic.cu

#include <iostream>
#include <cuda.h>

using namespace std;

int *host_A, *host_C1, *host_C2;       // host data
int *device_A, *device_C;              // results

// execute on device
__global__ void vecAddOne(int *A, int *C, int N)
{
   int i = blockIdx.x * blockDim.x + threadIdx.x;

   if( i < N )
      C[i] = A[i] + 1;
}

// execute on host
void vecAddOne_h(int *A1, int *C1, int N)
{
   for(int i = 0; i < N; i++)
      C1[i] = A1[i] + 1;
}

int main(int argc, char **argv)
{
   int n = 1024 * 1024; // number of threads (elements)
   int nBytes = n * sizeof(int);
   int block_size = 32;
   int block_number = n / block_size;

   // ===============================================================
   // CPU Memory settings

  printf("Allocating memory on host.\n");

   host_A = (int *) malloc(nBytes);
   host_C1 = (int *) malloc(nBytes);
   host_C2 = (int *) malloc(nBytes);

   // ===============================================================

   printf("Allocating memory on device.\n");

   cudaMalloc((void **) &device_A, n * sizeof(int));
   cudaMalloc((void **) &device_C, n * sizeof(int));

   // ===============================================================

   printf("Copying to device.\n");

   cudaMemcpy(device_A, host_A, n * sizeof(int), cudaMemcpyHostToDevice);

   // ===============================================================

   printf("Doing GPU Vector + 1 \n");

   vecAddOne<<<block_number, block_size>>>(device_A, device_C, n);
   cudaDeviceSynchronize();

   // ===============================================================

   printf("Doing a CPU Vector add & Copy to host\n");

   vecAddOne_h(host_A, host_C1, n);

   cudaMemcpy(host_C2, device_C, n * sizeof(int), cudaMemcpyDeviceToHost);

   // Compare Results

   printf("Compare Results\n");

   for(int i = 0; i < n; i++)
   {
       if(host_C1[i] != host_C2[i])
       {
           printf("Something Wrong ! \n");
           break;
       }
   }

   printf("Free resources");

   cudaFree(device_A);
   cudaFree(device_C);

   free(host_A);
   free(host_C1);
   free(host_C2);

   return 0;
}

Overwriting cudabasic.cu


In [ ]:
!nvcc -o cudabasic cudabasic.cu

In [ ]:
!./cudabasic

Allocating memory on host.
Allocating memory on device.
Copying to device.
Doing GPU Vector + 1 
Doing a CPU Vector add & Copy to host
Compare Results
Free resources

*  *  *
## nvprof:

It is a command-line profiler that can be used for
- quick checks
- profiling anything (no matter which language the CUDA kernel is written as long as it is launched using the CUDA runtime API or driver API)
- remote profiling (connect to the remote machine, using `ssh`, for example, and run your application under `nvprof`)

See also:

- Official doc: https://docs.nvidia.com/cuda/profiler-users-guide/index.html#nvprof
- https://devblogs.nvidia.com/cuda-pro-tip-nvprof-your-handy-universal-gpu-profiler/

**NVIDIA Visual Profiler (NVVP) and NVIDIA nvprof are deprecated!**

How to run: `nvprof [options] [application]
    [application-arguments]`

Help page: `nvprof --help`


In [ ]:
!nvprof --help

Usage: nvprof [options] [application] [application-arguments]
Options:
       --aggregate-mode <on|off>
                        Turn on/off aggregate mode for events and metrics specified
                        by subsequent "--events" and "--metrics" options. Those
                        event/metric values will be collected for each domain instance,
                        instead of the whole device. Allowed values:
                        	on - turn on aggregate mode (default)
                        	off - turn off aggregate mode

       --analysis-metrics
                        Collect profiling data that can be imported to Visual Profiler's
                        "analysis" mode. Note: Use "--export-profile" to specify
                        an export file.

       --annotate-mpi <off|openmpi|mpich>
                        Automatically annotate MPI calls with NVTX markers. Specify
                        the MPI implementation installed on your machine. Currently,
        

There are some modes of running `nvprof`.

**Summary mode**:
- default mode
- overview of the GPU kernels and memory copies in your application
- output a single result line for each kernel function and each type of CUDA memory copy/set performed by the application
- for each kernel, output
  - the total time of all instances of the kernel or type of memory copy
  - the average time
  - the minimum time
  - the maximum time
- support of CUDA dynamic parallelism:  if the app uses dynamic parallelism, the output will contain one column for the number of host-launched kernels and one for the number of device-launched kernels

*The time for a kernel* is the kernel execution time on the device

In [ ]:
!nvprof ./cudabasic

Allocating memory on host.
Allocating memory on device.
==457== NVPROF is profiling process 457, command: ./cudabasic
Copying to device.
Doing GPU Vector + 1 
Doing a CPU Vector add & Copy to host
Compare Results
Free resources==457== Profiling application: ./cudabasic
==457== Profiling result:
            Type  Time(%)      Time     Calls       Avg       Min       Max  Name
 GPU activities:   59.36%  1.9856ms         1  1.9856ms  1.9856ms  1.9856ms  [CUDA memcpy DtoH]
                   37.24%  1.2456ms         1  1.2456ms  1.2456ms  1.2456ms  [CUDA memcpy HtoD]
                    3.40%  113.73us         1  113.73us  113.73us  113.73us  vecAddOne(int*, int*, int)
      API calls:   93.85%  91.940ms         2  45.970ms  74.953us  91.865ms  cudaMalloc
                    5.25%  5.1401ms         2  2.5701ms  1.5640ms  3.5761ms  cudaMemcpy
                    0.35%  344.30us         2  172.15us  134.38us  209.92us  cudaFree
                    0.26%  252.83us         1  252.83us  252.83u

`--profile-api-trace none`: turn off API trace

In [ ]:
!nvprof --profile-api-trace none ./cudabasic

Allocating memory on host.
Allocating memory on device.
==482== NVPROF is profiling process 482, command: ./cudabasic
Copying to device.
Doing GPU Vector + 1 
Doing a CPU Vector add & Copy to host
Compare Results
Free resources==482== Profiling application: ./cudabasic
==482== Profiling result:
            Type  Time(%)      Time     Calls       Avg       Min       Max  Name
 GPU activities:   60.85%  1.8886ms         1  1.8886ms  1.8886ms  1.8886ms  [CUDA memcpy DtoH]
                   35.48%  1.1011ms         1  1.1011ms  1.1011ms  1.1011ms  [CUDA memcpy HtoD]
                    3.67%  114.05us         1  114.05us  114.05us  114.05us  vecAddOne(int*, int*, int)
No API activities were profiled.


*  *  *
*  *  *

**Kernel**: function that executes on device (GPU) and can be called from host (CPU)

- Functions must be declared with a qualifier
   - \_\_global\_\_: GPU kernel function launched by CPU, must return void
   - \_\_device\_\_: can be called from GPU functions
   - \_\_host\_\_: can be called from CPU functions (default)

In [ ]:
%%writefile cudaQual.cu

#include <stdio.h>

__device__ __host__ void hello()
{
  printf("Hello!\n");
}

__device__ void hiDeviceFunction(void)
{ printf("Hello! This is in hiDeviceFunction. \n");
  hello();
}

__global__ void helloCUDA(void)
{
  printf("Hello thread %d\n", threadIdx.x);
  hiDeviceFunction();
  hello();
}




int main()
{
  helloCUDA<<<1, 1>>>();
  cudaDeviceSynchronize();
  return 0;
}


Overwriting cudaQual.cu


In [ ]:
!nvcc -o cudaQual cudaQual.cu

In [ ]:
!./cudaQual

Hello thread 0
Hello! This is in hiDeviceFunction. 
Hello!
Hello!


Is the result value of the above code the value we expect ?

If the value we expected didn't come out, what would be the reason ?

In [ ]:
!nvprof ./cudaQual

==740== NVPROF is profiling process 740, command: ./cudaQual
Hello thread 0
Hello! This is in hiDeviceFunction. 
==740== Profiling application: ./cudaQual
==740== Warning: 1 records have invalid timestamps due to insufficient device buffer space. You can configure the buffer space using the option --device-buffer-size.
==740== Profiling result:
No kernels were profiled.
            Type  Time(%)      Time     Calls       Avg       Min       Max  Name
      API calls:   99.82%  92.215ms         1  92.215ms  92.215ms  92.215ms  cudaLaunchKernel
                    0.15%  142.78us       114  1.2520us     136ns  55.510us  cuDeviceGetAttribute
                    0.01%  13.231us         1  13.231us  13.231us  13.231us  cuDeviceGetName
                    0.01%  5.9930us         1  5.9930us  5.9930us  5.9930us  cuDeviceTotalMem
                    0.01%  5.0670us         1  5.0670us  5.0670us  5.0670us  cuDeviceGetPCIBusId
                    0.00%  1.7110us         3     570ns     189ns  1.

---
---

## Understanding Thread and Block

- blockIdx.x / blockIdx.y
- blockDim.x / blockDim.y
- threadIdx.x / threadIdx.y
- threadDim.x / threadDim.y


In [ ]:
%%writefile helloCUDA1.cu

// Create multiple blocks! --------------> 10 blocks
// Each block contains only one thread -----> 1 thread per block

#include <stdio.h>

__global__ void helloCUDA(void)
{
  printf("Hello thread %d in block %d\n", threadIdx.x, blockIdx.x);
}

int main()
{

  helloCUDA<<<10, 1>>>();

  cudaDeviceSynchronize();  // printf Wait until the function completes
  return 0;
}

Writing helloCUDA1.cu


In [ ]:
!nvcc -o helloCUDA1 helloCUDA1.cu

In [ ]:
!./helloCUDA1

Hello thread 0 in block 2
Hello thread 0 in block 7
Hello thread 0 in block 4
Hello thread 0 in block 9
Hello thread 0 in block 0
Hello thread 0 in block 3
Hello thread 0 in block 8
Hello thread 0 in block 5
Hello thread 0 in block 1
Hello thread 0 in block 6


In [ ]:
!nvprof ./helloCUDA1

==830== NVPROF is profiling process 830, command: ./helloCUDA1
Hello thread 0 in block 2
Hello thread 0 in block 7
Hello thread 0 in block 3
Hello thread 0 in block 8
Hello thread 0 in block 0
Hello thread 0 in block 5
Hello thread 0 in block 4
Hello thread 0 in block 1
Hello thread 0 in block 6
Hello thread 0 in block 9
==830== Profiling application: ./helloCUDA1
==830== Profiling result:
            Type  Time(%)      Time     Calls       Avg       Min       Max  Name
 GPU activities:  100.00%  100.93us         1  100.93us  100.93us  100.93us  helloCUDA(void)
      API calls:   99.68%  97.665ms         1  97.665ms  97.665ms  97.665ms  cudaLaunchKernel
                    0.15%  143.63us       114  1.2590us     145ns  56.938us  cuDeviceGetAttribute
                    0.14%  139.92us         1  139.92us  139.92us  139.92us  cudaDeviceSynchronize
                    0.02%  15.653us         1  15.653us  15.653us  15.653us  cuDeviceGetName
                    0.01%  5.7280us         1  5

In [ ]:
%%writefile helloCUDA2.cu

// Create one block! --------------> 1 Block
// Each block contains 10 threads -----> 10 threads per block

#include <stdio.h>

__global__ void helloCUDA(void)
{
  printf("Hello thread %d in block %d\n", threadIdx.x, blockIdx.x);
}

int main()
{

  helloCUDA<<<1, 10>>>();

  cudaDeviceSynchronize();  // printf Wait until the function completes
  return 0;
}

Writing helloCUDA2.cu


In [ ]:
!nvcc -o helloCUDA2 helloCUDA2.cu

In [ ]:
!./helloCUDA2

Hello thread 0 in block 0
Hello thread 1 in block 0
Hello thread 2 in block 0
Hello thread 3 in block 0
Hello thread 4 in block 0
Hello thread 5 in block 0
Hello thread 6 in block 0
Hello thread 7 in block 0
Hello thread 8 in block 0
Hello thread 9 in block 0


In [ ]:
!nvprof ./helloCUDA2

==974== NVPROF is profiling process 974, command: ./helloCUDA2
Hello thread 0 in block 0
Hello thread 1 in block 0
Hello thread 2 in block 0
Hello thread 3 in block 0
Hello thread 4 in block 0
Hello thread 5 in block 0
Hello thread 6 in block 0
Hello thread 7 in block 0
Hello thread 8 in block 0
Hello thread 9 in block 0
==974== Profiling application: ./helloCUDA2
==974== Profiling result:
            Type  Time(%)      Time     Calls       Avg       Min       Max  Name
 GPU activities:  100.00%  91.072us         1  91.072us  91.072us  91.072us  helloCUDA(void)
      API calls:   99.67%  94.849ms         1  94.849ms  94.849ms  94.849ms  cudaLaunchKernel
                    0.15%  146.46us         1  146.46us  146.46us  146.46us  cudaDeviceSynchronize
                    0.15%  138.71us       114  1.2160us     137ns  55.900us  cuDeviceGetAttribute
                    0.01%  11.242us         1  11.242us  11.242us  11.242us  cuDeviceGetName
                    0.01%  5.5080us         1  5

In [ ]:
%%writefile helloCUDA3.cu

#include <stdio.h>

__global__ void helloCUDA(void)
{
  printf("Hello thread %d in block %d\n", threadIdx.x, blockIdx.x);
}

int main()
{
  int n = 12;
  int blockDim = 4;            // Number of Threads within a Block
  int gridDim = n / blockDim;  // Number of Blocks in Grid

  // Thus, the total number of generated threads is blockDim * threadDim

  helloCUDA<<<gridDim, blockDim>>>();

  cudaDeviceSynchronize();
  return 0;
}

Writing helloCUDA3.cu


In [ ]:
!nvcc -o helloCUDA3 helloCUDA3.cu

In [ ]:
!./helloCUDA3

Hello thread 0 in block 2
Hello thread 1 in block 2
Hello thread 2 in block 2
Hello thread 3 in block 2
Hello thread 0 in block 0
Hello thread 1 in block 0
Hello thread 2 in block 0
Hello thread 3 in block 0
Hello thread 0 in block 1
Hello thread 1 in block 1
Hello thread 2 in block 1
Hello thread 3 in block 1


**GPU-trace mode**:
- timeline of all activities taking place on the GPU in chronological order
- for each kernel or memory copy, detailed information such as kernel parameters, shared memory usage and memory transfer throughput are shown
- the number shown in the square brackets after the kernel name correlates to the CUDA API that launched that kernel.
- `nvprof --print-gpu-trace`
- support of CUDA dynamic parallelism:
  - for host kernel launch, the kernel ID will be shown
  - for device kernel launch, the kernel ID, parent kernel ID and parent block will be shown

In [ ]:
!nvprof --print-gpu-trace ./helloCUDA3

==1077== NVPROF is profiling process 1077, command: ./helloCUDA3
Hello thread 0 in block 2
Hello thread 1 in block 2
Hello thread 2 in block 2
Hello thread 3 in block 2
Hello thread 0 in block 0
Hello thread 1 in block 0
Hello thread 2 in block 0
Hello thread 3 in block 0
Hello thread 0 in block 1
Hello thread 1 in block 1
Hello thread 2 in block 1
Hello thread 3 in block 1
==1077== Profiling application: ./helloCUDA3
==1077== Profiling result:
   Start  Duration            Grid Size      Block Size     Regs*    SSMem*    DSMem*           Device   Context    Stream  Name
233.82ms  90.560us              (3 1 1)         (4 1 1)        32        0B        0B     Tesla T4 (0)         1         7  helloCUDA(void) [125]

Regs: Number of registers used per CUDA thread. This number includes registers used internally by the CUDA driver and/or tools and can be more than what the compiler shows.
SSMem: Static shared memory allocated per CUDA block.
DSMem: Dynamic shared memory allocated per CUDA 

**API-trace mode:**
- timeline of all CUDA runtime and driver API calls invoked on the host in chronological order
- `--print-api-trace`

In [ ]:
%%writefile simpleCUDA.cu

#include <stdio.h>
__global__ void kernel1( int *a )
{
   int idx = blockIdx.x*blockDim.x + threadIdx.x;
   a[idx] = 7;          // output: 7 7 7 7   7 7 7 7   7 7 7 7   7 7 7 7
}

__global__ void kernel2( int *a )
{
 int idx = blockIdx.x*blockDim.x + threadIdx.x;
   a[idx] = blockIdx.x; // output: 0 0 0 0   1 1 1 1   2 2 2 2   3 3 3 3
}

__global__ void kernel3( int *a )
{
 int idx = blockIdx.x*blockDim.x + threadIdx.x;
   a[idx] = threadIdx.x;        // output: 0 1 2 3   1 2 3 4   0 1 2 3   0 1 2 3
}

int main()
{
  int *host_array;
  int *dev_array;

  host_array = (int *) malloc(sizeof(int)*16);
  cudaMalloc(&dev_array, sizeof(int)*16);
  cudaMemset(dev_array, 0, 16);

  kernel1<<<4, 4>>>(dev_array);

  cudaMemcpy(host_array, dev_array, sizeof(int)*16, cudaMemcpyDeviceToHost);

  for(int i = 0; i < 16; i++) printf(" %d ", host_array[i]);
  printf("\n");

  cudaMemset(dev_array, 0, 16);

  kernel2<<<4, 4>>>(dev_array);

  cudaMemcpy(host_array, dev_array, sizeof(int)*16, cudaMemcpyDeviceToHost);

  for(int i = 0; i < 16; i++) printf(" %d ", host_array[i]);
  printf("\n");

  cudaMemset(dev_array, 0, 16);

  kernel3<<<4, 4>>>(dev_array);

  cudaMemcpy(host_array, dev_array, sizeof(int)*16, cudaMemcpyDeviceToHost);

  for(int i = 0; i < 16; i++) printf(" %d ", host_array[i]);
  printf("\n");

  free(host_array);
  cudaFree(dev_array);
  cudaDeviceReset();
  return 0;
}

Writing simpleCUDA.cu


In [ ]:
!nvcc -o simpleCUDA simpleCUDA.cu

In [ ]:
!./simpleCUDA

 7  7  7  7  7  7  7  7  7  7  7  7  7  7  7  7 
 0  0  0  0  1  1  1  1  2  2  2  2  3  3  3  3 
 0  1  2  3  0  1  2  3  0  1  2  3  0  1  2  3 


In [ ]:
!nvprof --print-api-trace ./simpleCUDA

==2441== NVPROF is profiling process 2441, command: ./simpleCUDA
 7  7  7  7  7  7  7  7  7  7  7  7  7  7  7  7 
 0  0  0  0  1  1  1  1  2  2  2  2  3  3  3  3 
 0  1  2  3  0  1  2  3  0  1  2  3  0  1  2  3 
==2441== Profiling application: ./simpleCUDA
==2441== Profiling result:
   Start  Duration  Name
134.57ms  5.3540us  cuDeviceGetPCIBusId
146.79ms     944ns  cuDeviceGetCount
146.79ms     187ns  cuDeviceGetCount
147.14ms     931ns  cuDeviceGet
147.14ms  1.2480us  cuDeviceGetAttribute
147.20ms     693ns  cuDeviceGetAttribute
147.22ms     467ns  cuDeviceGetAttribute
147.29ms     748ns  cuModuleGetLoadingMode
147.34ms     345ns  cuDeviceGetCount
147.34ms     192ns  cuDeviceGet
147.34ms  10.991us  cuDeviceGetName
147.35ms  4.3490us  cuDeviceTotalMem
147.36ms     389ns  cuDeviceGetAttribute
147.36ms     168ns  cuDeviceGetAttribute
147.36ms     237ns  cuDeviceGetAttribute
147.36ms     268ns  cuDeviceGetAttribute
147.36ms     207ns  cuDeviceGetAttribute
147.36ms  23.837us  cuDeviceGetA

**Some useful `nvprof` arguments:**

- `--log-file file`: output into a file
- `--csv`: output is csv file
- `--analysis-metrics -o file.nvprof`: capture all of the GPU metrics that the Visual Profiler needs for its “guided analysis” mode
- `--print-summary-per-gpu`: print one summary per GPU if multiple CUDA capable devices are profiled
- `--query-events`: list of all available events on a particular NVIDIA GPU
- `--query-metrics`: list of all available metrics on a particular NVIDIA GPU
- `--events all`: collect all events available on each device
- `--metrics all`: collect all metrics available on each device
- `--timeout sec_num`: the CUDA application being profiled will be killed after the timeout
- `--concurrent-kernels off`: forces concurrent kernel executions to be serialized when a CUDA application is run with `nvprof`
- `--devices <device IDs>`: profile kernels run only on specific devices
- `--kernels <kernel filter>` where kernel filter is `<kernel name>` or `<context id/name>:<stream id/name>:<kernel name>:<invocation>`
- `--cpu-profiling on` (has restrictions)

**Event/metric Trace Mode:**

- event and metric values are shown for each kernel execution
- by default, event and metric values are aggregated across all units in the GPU (for example, multiprocessor specific events are aggregated across all multiprocessors on the GPU; turn off: `--aggregate-mode off`)

In [ ]:
!nvprof --log-file out.nvprof ./simpleCUDA

 7  7  7  7  7  7  7  7  7  7  7  7  7  7  7  7 
 0  0  0  0  1  1  1  1  2  2  2  2  3  3  3  3 
 0  1  2  3  0  1  2  3  0  1  2  3  0  1  2  3 


# Nsight Systems

- Statistical sampling profiler with tracing features
- Some terms:
  - *Target* - device on which profiling happens
  - *Host* - computer on which the user works and controls the profiling session
  - Profiling - process of collecting any performance data
  - Profilee - app under investigation during the profiling session
  - Backtraces - call stack of active threads
  - Sampling - process of periodically stopping the profilee, typically to collect backtraces, which allows you to understand statistically how much time is spent in each function
  - Tracing - process of collecting precise information about various activities happening in the profilee or in the system (for example, profilee API execution may be traced providing the exact time and duration of a function call)

System requirements: https://docs.nvidia.com/nsight-systems/InstallationGuide/index.html#installation-guide

Installation in Google Collab ([Инструкция на Stack Overflow](https://stackoverflow.com/questions/76784746/how-to-use-nsys-in-google-colab))

In [ ]:
!wget https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64/nsight-systems-2023.2.3_2023.2.3.1001-1_amd64.deb
!apt update
!apt install ./nsight-systems-2023.2.3_2023.2.3.1001-1_amd64.deb
!apt --fix-broken install

--2024-02-14 09:25:59--  https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64/nsight-systems-2023.2.3_2023.2.3.1001-1_amd64.deb
Resolving developer.download.nvidia.com (developer.download.nvidia.com)... 152.199.20.126
Connecting to developer.download.nvidia.com (developer.download.nvidia.com)|152.199.20.126|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 317705436 (303M) [application/x-deb]
Saving to: ‘nsight-systems-2023.2.3_2023.2.3.1001-1_amd64.deb’

nsight-systems-2023 100%[===================>] 302.99M   155MB/s    in 1.9s    

2024-02-14 09:26:01 (155 MB/s) - ‘nsight-systems-2023.2.3_2023.2.3.1001-1_amd64.deb’ saved [317705436/317705436]

Hit:1 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:2 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [119 kB]
Hit:3 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Get:4 http://security.ubuntu.com/ubuntu jammy-security InRelease [110 kB]
Get:5 https://cloud.r-project.org

**Command Line Options:**
- `nsys [global_option]`
- `nsys [command_switch][optional command_switch_options][application] [optional application_options]`

*Short v.s. long options: *
- `-s process-tree`
- `--sample=process-tree`

**Global options:**
- `--help / -h`
- `--version / -v`

In [ ]:
!nsys --help

 usage: nsys [--version] [--help] <command> [<args>] [application] [<application args>]

 The most commonly used nsys commands are:
	profile       Run an application and capture its profile into a QDSTRM file.
	launch        Launch an application ready to be profiled.
	start         Start a profiling session.
	stop          Stop a profiling session and capture its profile into a QDSTRM file.
	cancel        Cancel a profiling session and discard any collected data.
	service       Launch the Nsight Systems data service.
	stats         Generate statistics from an existing nsys-rep or SQLite file.
	status        Provide current status of CLI or the collection environment.
	shutdown      Disconnect launched processes from the profiler and shutdown the profiler.
	sessions list List active sessions.
	export        Export nsys-rep file into another format.
	analyze       Identify optimization opportunities in a nsys-rep or SQLITE file.
	recipe        Run a recipe for multi-node analysis.
	nvpr

**Profile Command**

`nsys [global-options] profile [options] <application> [application-arguments]`

[Profile options](https://docs.nvidia.com/nsight-systems/UserGuide/index.html#cli-profile-command-switch-options)






In [ ]:
!nsys export --help


usage: nsys export [<args>] [nsys-rep-file]

	-f, --force-overwrite=

	   Possible values are 'true' or 'false'.
	   If true, overwrite all existing result files with same output filename
	   (QDSTRM, nsys-rep, SQLITE, HDF, TEXT, ARROW, JSON).
	   Default is 'false'.

	-h, --help=[<tag>]

	   Print the command's help menu. The switch can take one optional
	   argument that will be used as a tag. If a tag is provided, only options
	   relevant to the tag will be printed.
	   The available help menu tags for this command are:

	   export, output, and type.

	-l, --lazy=

           Possible values are 'true' or 'false'.
           Controls if table creation is lazy or not. Lazy table creation will
           only create a table if it contains data. This affects SQLite, HDF5,
           and Arrow exports only. Default is 'true', although this is
           likely to change in a future release.

	-o, --output=

           Path to results file.
           Default is name of input file with

In [ ]:
!nsys profile --help


usage: nsys profile [<args>] [application] [<application args>]

	-b, --backtrace=

	   Possible values are 'lbr', 'fp', 'dwarf', or 'none'.
	   Select the backtrace method to use while sampling.
	   Select 'none' to disable backtrace collection.
	   Default is 'lbr'.

	-c, --capture-range=

	   Possible values are none, cudaProfilerApi, nvtx, hotkey.
	   When '-c cudaProfilerApi' is used, profiling will start only when cudaProfilerStart API is
	   invoked in the application.
	   When '-c nvtx' is used, profiling will start only when the specified NVTX range is
	   started in the application.
	   When '-c hotkey' is used, profiling will start only when the hotkey
	   set by '--hotkey-capture' is pressed in the application. This works for graphic apps only.
	   Note that you must enable CUDA or NVTX tracing of the target application
	   for '-c cudaProfilerApi' or '-c nvtx' to work.
	   When '-capture-range none' is used, cudaProfilerStart/Stop APIs and hotkey will 
	   be ignored and 

**Useful options: **
- `-t [cuda/nvtx/mpi/cudnn/opengl/openacc/none...]` - select APIs to trace (multiple APIs can be selected, separated by commas only)
- `-d [number]` - collection duration in sec
- `-o [file_name]` - generate output report file
- `--gpu-metrics-device=[0/all...]` - collect GPUs metrics from specified device
- `--gpu-metrics-set=[tu10x-gfxt/...]` - specify metric set for GPU Metrics sampling
- `--sample=[process-tree/system-wide/none]` - collect CPU IP/backtrace samples
- `--cpu-core-events=help` - get list of available CPU core events
- `--stats=true` - generate summary statistics after the collection

Profile Python script: `nsys profile python program.py`



*Default analysis run*:

`nsys profile <application> [application-arguments]`
- start collecting immediately and end collection when the application stops
- trace CUDA, OpenGL, NVTX, and OS runtime libraries APIs
- collect CPU sampling information and thread scheduling information

**nvprof Command**:
- `nsys nvprof [options]`
- help former `nvprof` users transition to nsys
- not all the commands from original `nvprof` are available in `nsys`!
- [options](https://docs.nvidia.com/nsight-systems/UserGuide/index.html#cli-nvprof-command-switch-options)

In [ ]:
%%writefile example.py

import numpy as np

def main():
    for i in range(10):
        x = np.array(range(10**7))
        y = np.array(np.random.uniform(0, 100, size=(10**(8))))


main()

Writing example.py


In [ ]:
!nsys profile --stats=true --python-sampling=true python example.py

Generating '/tmp/nsys-report-b102.qdstrm'
[1/8] [========================100%] report1.nsys-rep
[2/8] [========================100%] report1.sqlite
[3/8] Executing 'nvtx_sum' stats report
SKIPPED: No data available.
[4/8] Executing 'osrt_sum' stats report

 Time (%)  Total Time (ns)  Num Calls  Avg (ns)   Med (ns)   Min (ns)  Max (ns)   StdDev (ns)           Name         
 --------  ---------------  ---------  ---------  ---------  --------  ---------  -----------  ----------------------
     89.6      417,666,266      3,060  136,492.2  132,290.0    81,145  1,927,466     50,824.6  munmap                
      4.5       21,131,650      3,071    6,881.0    6,363.0     4,577     94,738      3,093.2  mmap64                
      3.2       14,891,475         62  240,185.1   16,495.5     8,312  1,831,022    415,088.4  pthread_cond_wait     
      1.1        4,909,445         21  233,783.1    6,618.0     1,097  1,695,791    485,526.4  pthread_cond_timedwait
      0.7        3,327,386        3

In [ ]:
!pip install -U py-boost

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.4/58.4 kB 2.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 28.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 198.7/198.7 kB 24.5 MB/s eta 0:00:00


In [ ]:
%%writefile example2.py

import numpy as np
import pandas as pd
import cupy as cp
from py_boost.gpu.utils import *
from astropy.table import Table
from cupyx.profiler import benchmark

histogram_kernel_idx_elw = cp.ElementwiseKernel(
    """
    uint64 i_, uint64 j_, uint64 k_,
    uint64 kk,

    raw uint64 jj,
    raw bool padded_bool_indexer,

    raw float32 target,
    raw T arr,
    raw int32 nodes,

    uint64 hlen,
    uint64 flen,
    uint64 length,
    uint64 feats,
    uint64 nout
    """,
    'raw float32 hist',

    """
    unsigned int feat_4t = arr[i_ * feats + j_];
    int d;
    int j;
    int val;
    int pos;
    float *x_ptr;
    float y = target[i_ * nout + k_];

    for (d = 0; d < 4; d++) {

        pos = (i_ + d) % 4;

        if (padded_bool_indexer[j_ * 4 + pos]) {

            val = (feat_4t >> (8 * pos)) % 256;
            j = jj[j_ * 4 + pos];
            x_ptr = &hist[0] +  kk * hlen + nodes[i_] * flen + j * length + val;
            atomicAdd(x_ptr, y);
        }
    }

    """,

    'histogram_kernel_idx')


def fill_histogram_tmp(res, arr, target, nodes, col_indexer, row_indexer, out_indexer, func='elw'):
    """Fill the histogram res

    Args:
        res: cp.ndarray, histogram of zeros, shape (n_out, n_nodes, n_features, n_bins)
        arr: cp.ndarray, features array, shape (n_data, n_features)
        target: cp.ndarray, values to accumulate, shape (n_data, n_out)
        nodes: cp.ndarray, tree node indices, shape (n_data, )
        col_indexer: cp.ndarray, indices of features to accumulate
        row_indexer: cp.ndarray, indices of rows to accumulate
        out_indexer: cp.ndarray, indices of outputs to accumulate
        func: numeric flag to choose the kernel that will be used in histogram calculations

    Returns:

    """
    # define data split for kernel launch
    nout, nnodes, nfeats, nbins = res.shape

    # padded array of 4 feature tuple
    arr_4t = arr.base.view(dtype=cp.uint32)
    pfeats = arr_4t.shape[1]

    # create 4 feats tuple indexer
    padded_bool_indexer = cp.zeros((arr.base.shape[1],), dtype=cp.bool_)
    padded_col_indexer = cp.zeros((arr.base.shape[1],), dtype=cp.uint64)
    tuple_indexer = cp.zeros((arr_4t.shape[1],), dtype=cp.bool_)

    feature_grouper_kernel(col_indexer, padded_bool_indexer, tuple_indexer, padded_col_indexer)
    tuple_indexer = cp.arange(arr_4t.shape[1], dtype=cp.uint64)[tuple_indexer]

    fb = nfeats * nbins
    nfb = nnodes * fb

    magic_constant = 2 ** 19  # optimal value for my V100

    # split features
    nsplits = math.ceil(nfb / magic_constant)
    # first split by feats
    feats_batch = math.ceil(pfeats / nsplits)
    # split by features
    if feats_batch == nfeats:
        out_batch = magic_constant // nfb
    else:
        out_batch = 1

    ri = row_indexer[:, cp.newaxis, cp.newaxis]
    ti = tuple_indexer[cp.newaxis, :, cp.newaxis]
    oi = out_indexer[cp.newaxis, cp.newaxis, :]

    nrows = ri.shape[0]

    oii = cp.arange(oi.shape[2], dtype=cp.uint64)[cp.newaxis, cp.newaxis, :]

    if func == 'ser2' or func == 'ser3':
        with cp.cuda.Device(0):
            res0 = cp.zeros(res.shape, dtype=cp.float32)
        with cp.cuda.Device(1):
            res1 = cp.zeros(res.shape, dtype=cp.float32)
            ri_d1 = ri.copy()
            padded_col_indexer_d1  = padded_col_indexer.copy()
            padded_bool_indexer_d1 = padded_bool_indexer.copy()
            target_d1 = target.copy()
            arr_4t_d1 = arr_4t.copy()
            nodes_d1  = nodes.copy()
            nfb_d1    = nfb
            fb_d1     = fb
            nbins_d1  = nbins
            nout_d1   = nout

    for j in range(0, pfeats, feats_batch):
        ti_ = ti[:, j: j + feats_batch]

        for k in range(0, nout, out_batch):
            oi_ = oi[..., k: k + out_batch]
            oii_ = oii[..., k: k + out_batch]

            if func == 'elw':
                # Use original Anton's solution
                histogram_kernel_idx_elw(ri, ti_, oi_,
                                     oii_,
                                     padded_col_indexer,
                                     padded_bool_indexer,
                                     target,
                                     arr_4t,
                                     nodes,
                                     nfb, fb, nbins, arr_4t.shape[1], nout,
                                     res, block_size=1024)
            if func == 'ser':
                histogram_kernel_idx_ser(ri, ti_, oi_,
                                         oii_,
                                         padded_col_indexer,
                                         padded_bool_indexer,
                                         target,
                                         arr_4t,
                                         nodes,
                                         nfb, fb, nbins, arr_4t.shape[1], nout,
                                         res, block_size=1024 )
            if func == 'ser2':
                with cp.cuda.Device(0):
                    histogram_kernel_idx_ser(ri[0:nrows//2], ti_, oi_,
                                             oii_,
                                             padded_col_indexer,
                                             padded_bool_indexer,
                                             target,
                                             arr_4t,
                                             nodes,
                                             nfb, fb, nbins, arr_4t.shape[1], nout,
                                             res0, block_size=1024 )

                with cp.cuda.Device(1):
                    ti_d1  = ti_.copy()
                    oi_d1  = oi_.copy()
                    oii_d1 = oii_.copy()
                    histogram_kernel_idx_ser(ri_d1[nrows//2:], ti_d1, oi_d1,
                                             oii_d1,
                                             padded_col_indexer_d1,
                                             padded_bool_indexer_d1,
                                             target_d1,
                                             arr_4t_d1,
                                             nodes_d1,
                                             nfb_d1, fb_d1, nbins_d1, arr_4t_d1.shape[1], nout_d1,
                                             res1, block_size=1024 )
            if func == 'ser2_target':
                with cp.cuda.Device(0):
                    histogram_kernel_idx_ser(ri, ti_, oi_,
                                             oii_,
                                             padded_col_indexer,
                                             padded_bool_indexer,
                                             target[0:nrows//2],
                                             arr_4t,
                                             nodes,
                                             nfb, fb, nbins, arr_4t.shape[1], nout,
                                             res, block_size=1024 )
                with cp.cuda.Device(1):
                    ti_d1  = ti_.copy()
                    oi_d1  = oi_.copy()
                    oii_d1 = oii_.copy()
                    histogram_kernel_idx_ser(ri_d1, ti_d1, oi_d1,
                                             oii_d1,
                                             padded_col_indexer_d1,
                                             padded_bool_indexer_d1,
                                             target_d1[nrows//2:],
                                             arr_4t_d1,
                                             nodes_d1,
                                             nfb_d1, fb_d1, nbins_d1, arr_4t_d1.shape[1], nout_d1,
                                             res1, block_size=1024)

    if func == 'ser2' or func == 'ser2_target':
        with cp.cuda.Device(0):
            res[:] = res0 + res1

    return


def sample_idx(n, sample):
    # THIST FUNCTION GENERATES IDS USED
    # IN THE HISTOGRAM CALCULATIONS

    idx = cp.arange(n, dtype=cp.uint64)
    sl = cp.random.rand(n) < sample

    return cp.ascontiguousarray(idx[sl])


def generate_input( n_rows, n_cols, n_out, max_bin, nnodes,
                    colsample=0.8, subsample=0.8, outsample=1.0, verbose=False, seed=42):
    # THIS FUNCTION GENERATES ALL INPUT
    # ARRAYS, REQUIRED BY THE HISTOGRAM
    # FUNCTION IN PY-BOOST
    # Input:
    # n_rows   - number of rows in the input array
    # n_cols   - number of cols in the input array
    # n_out    - number of ???? in the output array
    # max_bins - number of histogram bins (can't be >256 really)
    # nnodes   - ????

    np.random.seed(seed)
    features_cpu = np.random.randint(0, max_bin, size=(n_rows, n_cols)).astype(np.uint8)
    features_gpu = pad_and_move(features_cpu)
    cp.random.seed(seed)
    targets_gpu  = cp.random.rand(n_rows, n_out).astype(np.float32)
    cp.random.seed(seed)
    nodes_gpu    = cp.random.randint(0, nnodes, size=(n_rows, )).astype(np.int32)
    cp.random.seed(seed)

    if verbose == True:
        print('Initial CPU features shape: {}'.format(features_cpu.shape))
        print('Padded  GPU features shape: {}'.format(features_gpu.shape))
        print('Nodes   GPU vector   shape: {}'.format(nodes_gpu.shape   ))
        print('Targets GPU array    shape: {}'.format(targets_gpu.shape ))

    row_indexer = sample_idx(n_rows, subsample)
    col_indexer = sample_idx(n_cols, colsample)
    out_indexer = sample_idx(n_out, outsample)

    if verbose == True:
        print('Sampled rows shape:    {}'.format(row_indexer.shape))
        print('Sampled columns shape: {}'.format(col_indexer.shape))
        print('Sampled output shape:  {}'.format(out_indexer.shape))

    nout   = out_indexer.shape[0]
    nfeats = col_indexer.shape[0]

    # Anton's function takes the following input arguments + the empty array to
    # store the resulting histogram bins (comes in the first position)
    # input: res, X, Y, nodes, col_indexer, row_indexer, out_indexer

    res    = cp.zeros((nout, nnodes, nfeats, max_bin), dtype=cp.float32)
    params = (res, features_gpu, targets_gpu, nodes_gpu, col_indexer, row_indexer, out_indexer)

    if verbose == True:
        true_res = nfeats * targets_gpu[row_indexer].sum()
        print ('Sum of the resulting histogram must be {} ({}/2={})'.format( true_res, true_res, true_res/2 ))
    return params


# Original Anton's code on 1 GPU
input_params = generate_input(n_rows=pow(10,6),n_cols=99,n_out=10,max_bin=256,nnodes=32,verbose=False)
tau0 = benchmark( fill_histogram_tmp, (*input_params, 'elw'), n_repeat=1000, n_warmup=10 )

print (tau0)

Writing example2.py


In [ ]:
!nsys profile --stats=true --python-sampling=true python example2.py

fill_histogram_tmp  :    CPU:  1799.232 us   +/- 582.253 (min:  1295.151 / max:  6667.247) us     GPU-0: 38804.605 us   +/- 918.958 (min: 35661.247 / max: 41830.463) us
Generating '/tmp/nsys-report-7b6a.qdstrm'
[1/8] [========================100%] report3.nsys-rep
[2/8] [========================100%] report3.sqlite
[3/8] Executing 'nvtx_sum' stats report
SKIPPED: No data available.
[4/8] Executing 'osrt_sum' stats report

 Time (%)  Total Time (ns)  Num Calls    Avg (ns)      Med (ns)     Min (ns)    Max (ns)    StdDev (ns)            Name         
 --------  ---------------  ---------  ------------  -------------  ---------  -----------  ------------  ----------------------
     97.2   51,119,439,942        521  98,117,927.0  100,127,600.0      3,512  105,803,255  13,997,137.8  poll                  
      0.9      473,738,222      3,788     125,062.9        1,807.5        303   45,309,071     976,399.1  read                  
      0.8      418,550,047  6,239,131          67.1       

**Analyze command**:
- `nsys analyze file.nsys-rep`
- post process existing Nsight Systems result, either in .nsys-rep or SQLite format, to generate expert systems report

In [ ]:
!nsys analyze report3.nsys-rep


NOTICE: Existing SQLite export found: report3.sqlite
        It is assumed file was previously exported from: report3.nsys-rep
        Consider using --force-export=true if needed.

Processing [report3.sqlite] with [/opt/nvidia/nsight-systems/2023.2.3/host-linux-x64/rules/cuda_memcpy_async.py]... 

 ** CUDA Async Memcpy with Pageable Memory (cuda_memcpy_async):

There were no problems detected related to memcpy operations using pageable
memory.

Processing [report3.sqlite] with [/opt/nvidia/nsight-systems/2023.2.3/host-linux-x64/rules/cuda_memcpy_sync.py]... 

 ** CUDA Synchronous Memcpy (cuda_memcpy_sync):

The following are synchronous memory transfers that block the host. This does
not include host to device transfers of a memory block of 64 KB or less.

Suggestion: Use cudaMemcpy*Async() APIs instead.

 Duration (ns)    Start (ns)    Src Kind  Dst Kind  Bytes (MB)   PID    Device ID  Context ID  Stream ID      API Name    
 -------------  --------------  --------  --------  ------

## Export CLI output to visual profiler

**Export**

`!nvprof --export-profile filename app`

`!nsys nvprof --output=filename app`

`!nvprof --import-profile filename`

[Option desription](https://docs.nvidia.com/cuda/profiler-users-guide/index.html#io-options)

[How to import into nvprof visual profiler (nvvp)](https://docs.nvidia.com/cuda/profiler-users-guide/index.html#import-session):

- Click *Import* option in the *File* menu
- Can import single or multiple `nvprof` output files

**How to import into the Nsight Systems GUI:**

- The CLI and host GUI versions must match to import a `.qdstrm` file successfully; the host GUI is backward compatible only with `.nsys-rep` files.

- `File -> Import -> .qdstrm file`

- *The import of really large, multi-gigabyte, `.qdstrm` files may take up all of the memory on the host computer and lock up the system*

In [ ]:
!nvcc -o cudabasic cudabasic.cu

In [ ]:
!nvprof --export-profile result ./cudabasic

Allocating memory on host.
Allocating memory on device.
==15698== NVPROF is profiling process 15698, command: ./cudabasic
Copying to device.
Doing GPU Vector + 1 
Doing a CPU Vector add & Copy to host
Compare Results
Free resources==15698== Generated result file: /content/result


In [ ]:
!nvprof --import-profile result

======== Profiling result:
            Type  Time(%)      Time     Calls       Avg       Min       Max  Name
 GPU activities:   60.60%  1.9154ms         1  1.9154ms  1.9154ms  1.9154ms  [CUDA memcpy DtoH]
                   35.80%  1.1317ms         1  1.1317ms  1.1317ms  1.1317ms  [CUDA memcpy HtoD]
                    3.60%  113.70us         1  113.70us  113.70us  113.70us  vecAddOne(int*, int*, int)
      API calls:   94.38%  93.486ms         2  46.743ms  70.340us  93.415ms  cudaMalloc
                    4.82%  4.7730ms         2  2.3865ms  1.4441ms  3.3289ms  cudaMemcpy
                    0.33%  326.93us         2  163.47us  124.69us  202.25us  cudaFree
                    0.19%  191.70us         1  191.70us  191.70us  191.70us  cudaLaunchKernel
                    0.13%  131.85us       114  1.1560us     135ns  52.442us  cuDeviceGetAttribute
                    0.11%  113.61us         1  113.61us  113.61us  113.61us  cudaDeviceSynchronize
                    0.01%  11.774us       

# Occupancy

[Reference](https://docs.nvidia.com/gameworks/content/developertools/desktop/analysis/report/cudaexperiments/kernellevel/achievedoccupancy.htm)

- CUDA groups adjacent threads within a block into **warps**

- A **warp** is considered **active** from the time its threads begin executing to the time when all threads in the warp have exited from the kernel

- There is a **maximum number of warps** which can be concurrently active on a Streaming Multiprocessor (SM) (depends on he launch configuration, compile options for the kernel, and device capabilities)

- **Occupancy** is the ratio of active warps on an SM to the maximum number of active warps supported by the SM

- **Low occupancy** results in poor instruction issue efficiency, because there are not enough eligible warps to hide latency between dependent instructions

- When occupancy is at a sufficient level to hide latency, increasing it further **may degrade performance** due to the reduction in resources per thread

- An **early step of kernel performance analysis** should be to check occupancy and observe the effects on kernel execution time when running at different occupancy levels

## Theoretical Occupancy

- Each block of a kernel launch gets distributed to one of the SMs for execution

- A **block** is considered **active** from the time its warps begin executing to the time when all warps in the block have exited from the kernel

- **(upper limit for active warps) = (upper limit for active blocks) \* (number of warps per block)**

- The number of blocks which can execute concurrently on an SM is limited by the factors listed below

- Then how  we can increase the upper limit for active warps:

  - increase the number of warps per block (defined by block dimension)

  - or change the factors (see below) limiting how many blocks can fit on an SM to allow more active blocks

- **The factors limiting the number of concurrently active warps**:

  - *warps per SM*:

    -  The SM has a maximum number of warps that can be active at once

    - from the definition: occupancy is 100% if the number of active warps equals the maximum

    - If this factor is limiting active blocks, occupancy cannot be increased

    - [Example](https://docs.nvidia.com/gameworks/content/developertools/desktop/analysis/report/cudaexperiments/kernellevel/achievedoccupancy.htm)

  - *blocks per SM*:

    - The SM has a maximum number of blocks that can be active at once

    - If occupancy is below 100% and this factor is limiting active blocks, it means each block does not contain enough warps to reach 100% occupancy when the device's active block limit is reached
    
    - Occupancy can be increased by increasing block size

    - [Example](https://docs.nvidia.com/gameworks/content/developertools/desktop/analysis/report/cudaexperiments/kernellevel/achievedoccupancy.htm)

  - *registers per SM*:

    - The SM has a set of registers shared by all active threads

    -  If this factor is limiting active blocks, it means the number of registers per thread allocated by the compiler can be reduced to increase

    - The performance gain from improved latency hiding due to increased occupancy may be outweighed by the performance loss of having fewer registers per thread, and spilling to local memory more often

    - The best-performing balance of occupancy and registers per thread can be found experimentally by tracing the kernel compiled with different numbers of registers per thread

  - *shared Memory per SM*:

    - The SM has a fixed amount of shared memory shared by all active threads

    - If this factor is limiting active blocks, it means the shared memory needed per thread can be reduced to increase occupancy

    - (Shared memory per thread) = ("static shared memory") + ("dynamic shared memory"), where

      - "static shared memory" is the total size needed for all \_\_shared\_\_ variables

      - "dynamic shared memory" is the amount of shared memory specified as a parameter to the kernel launch

    - For some CUDA devices, the amount of shared memory per SM is configurable, trading between shared memory size and L1 cache size:

      - If such a GPU is configured to use more L1 cache and shared memory is the limiting factor for occupancy, then occupancy can also be increased by choosing to use less L1 cache and more shared memory

## Achieved Occupancy


- Theoretical occupancy shows the upper bound active warps on an SM, but the true number of active warps varies over the duration of the kernel, as warps begin and end

- A SM contain one or more warp schedulers

- Each warp scheduler attempts to issue instructions from a warp on each clock cycle

- To sufficiently hide latencies between dependent instructions, each scheduler must have at least one warp eligible to issue an instruction every clock cycle

- Maintaining as many active warps as possible (a high occupancy) throughout the execution of the kernel helps to avoid situations where all warps are stalled and no instructions are issued

- **Achieved occupancy** is measured on each warp scheduler using hardware performance counters to count the number of active warps on that scheduler every clock cycle

- These counts are then summed across all warp schedulers on each SM and divided by the clock cycles the SM is active to find the average active warps per SM

- Dividing by the SM's maximum supported number of active warps gives the **achieved occupancy per SM averaged over the duration of the kernel**

- Averaging across all SMs gives the **overall achieved occupancy**

### Causes of Low Achieved Occupancy

- *Achieved occupancy <= theoretical occupancy*

- **Steps to increase achieved occupancy**:

  1) increase theoretical occupancy by adjusting the limiting factors;

  2) check if the achieved value is close to the theoretical value:
  
    - (achieved occupancy <= theoretical occupancy)  when the theoretical number of active warps is not maintained for the full time the SM is active; it occurs in the situations:

      - **unbalanced workload within blocks**:

        - If warps within a block do not all execute for the same amount of time, the **workload** is said to be **unbalanced** <=> fewer active warps at the end of the kernel, which is a problem known as **"tail effect"**

        - **Best solution** is to try having a more balanced workload among the warps in each block

      - **unbalanced workload across blocks:**

        - Blocks within a grid do not all execute for the same amount of time

        - The efficiency of the device can be improved without having to change to a more balanced workload

        - Launching more blocks will allow new blocks to begin as others finish, meaning the tail effect does not occur inside every block, but only at the end of the kernel

        - If there are not more blocks to launch, running concurrent kernels with similar block properties can achieve the same effect

      - **too few blocks launched:**

        - ("Full wave") = (number of SMs on the device) \* (maximum active blocks per SM)

        - Launching less than a full wave results in low achieved occupancy

        - [Example](https://docs.nvidia.com/gameworks/content/developertools/desktop/analysis/report/cudaexperiments/kernellevel/achievedoccupancy.htm)



## Charts & Analysis

- varying block size
- varying register count
- varying share memory usage
- achieved occupancy per SM

[Charts](https://docs.nvidia.com/gameworks/content/developertools/desktop/analysis/report/cudaexperiments/kernellevel/achievedoccupancy.htm#Charts)

[What to do](https://docs.nvidia.com/gameworks/content/developertools/desktop/analysis/report/cudaexperiments/kernellevel/achievedoccupancy.htm#Analysis)